In [3]:
#Custom Seeds with the WaterShed Algorithm
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
road = cv2.imread('DATA/road_image.jpg')

In [5]:
road_copy = np.copy(road)

In [6]:
#plt.imshow(road)
#no color conv as we dontuse matplotlib using opencv

In [7]:
road.shape[:2]

(600, 800)

In [8]:
marker_image = np.zeros(road.shape[:2],dtype=np.int32)

In [9]:
segments = np.zeros(road.shape,dtype = np.uint8)

In [10]:
marker_image.shape

(600, 800)

In [11]:
segments.shape

(600, 800, 3)

In [12]:
#Create colors for Markers
from matplotlib import cm

In [13]:
cm.tab10(0)

(0.12156862745098039, 0.4666666666666667, 0.7058823529411765, 1.0)

In [14]:
np.array(cm.tab10(0))

array([0.12156863, 0.46666667, 0.70588235, 1.        ])

In [15]:
np.array(cm.tab10(0))[:3]

array([0.12156863, 0.46666667, 0.70588235])

In [16]:
np.array(cm.tab10(0))[:3]*255

array([ 31., 119., 180.])

In [17]:
x = np.array(cm.tab10(0))[:3]*255

In [18]:
x.astype(int)

array([ 31, 119, 180])

In [19]:
tuple(x.astype(int))

(31, 119, 180)

In [20]:
#Let's make a function for all those steps
def create_rgb(i):
    x = np.array(cm.tab10(i))[:3]*255
    return tuple(x)

In [21]:
colors = []

In [22]:
# One color for each single digit
for i in range(10):
    colors.append(create_rgb(i))

In [23]:
# Numbers 0-9
n_markers = 10

In [24]:
#Setting Up Callback Function
# Default settings
current_marker = 1
marks_updated = False

In [25]:
def mouse_callback(event, x, y, flags, param):
    global marks_updated 

    if event == cv2.EVENT_LBUTTONDOWN:
        
        # TRACKING FOR MARKERS
        cv2.circle(marker_image, (x, y), 10, (current_marker), -1)
        
        # DISPLAY ON USER IMAGE
        cv2.circle(road_copy, (x, y), 10, colors[current_marker], -1)
        marks_updated = True


In [ ]:
cv2.namedWindow('Road Image')
cv2.setMouseCallback('Road Image', mouse_callback)

while True:
    
    # SHow the 2 windows
    cv2.imshow('WaterShed Segments', segments)
    cv2.imshow('Road Image', road_copy)
        
        
    # Close everything if Esc is pressed
    k = cv2.waitKey(1)

    if k == 27:
        break
    # Clear all colors and start over if 'c' is pressed
    elif k == ord('c'):
        road_copy = road.copy()
        marker_image = np.zeros(road.shape[0:2], dtype=np.int32)
        segments = np.zeros(road.shape,dtype=np.uint8)
        
    # If a number 0-9 is chosen index the color
    elif k > 0 and chr(k).isdigit():
        # chr converts to printable digit
        
        current_marker  = int(chr(k))
        
        # CODE TO CHECK INCASE USER IS CARELESS
#         n = int(chr(k))
#         if 1 <= n <= n_markers:
#             current_marker = n
    
    # If we clicked somewhere, call the watershed algorithm 
    #on our chosen markers
    if marks_updated:
        
        marker_image_copy = marker_image.copy()
        cv2.watershed(road, marker_image_copy)
        
        segments = np.zeros(road.shape,dtype=np.uint8)
        
        for color_ind in range(n_markers):
            segments[marker_image_copy == (color_ind)] = colors[color_ind]
        
        marks_updated = False
        
cv2.destroyAllWindows()